https://www.kaggle.com/ruchi798/movies-on-netflix-prime-video-hulu-and-disney

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

In [2]:
rating_db = pd.read_csv("MoviesOnStreamingPlatforms_updated.csv")

In [3]:
rating_db.head()

,Unnamed: 0,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Directors,Genres,Country,Language,Runtime
0,0,1,Inception,2010,13+,8.8,87%,1,0,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0
1,1,2,The Matrix,1999,18+,8.7,87%,1,0,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136.0
2,2,3,Avengers: Infinity War,2018,13+,8.5,84%,1,0,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English,149.0
3,3,4,Back to the Future,1985,7+,8.5,96%,1,0,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English,116.0
4,4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1,0,1,0,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian,161.0


In [4]:
rating_db.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16744 entries, 0 to 16743
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       16744 non-null  int64  
 1   ID               16744 non-null  int64  
 2   Title            16744 non-null  object 
 3   Year             16744 non-null  int64  
 4   Age              7354 non-null   object 
 5   IMDb             16173 non-null  float64
 6   Rotten Tomatoes  5158 non-null   object 
 7   Netflix          16744 non-null  int64  
 8   Hulu             16744 non-null  int64  
 9   Prime Video      16744 non-null  int64  
 10  Disney+          16744 non-null  int64  
 11  Type             16744 non-null  int64  
 12  Directors        16018 non-null  object 
 13  Genres           16469 non-null  object 
 14  Country          16309 non-null  object 
 15  Language         16145 non-null  object 
 16  Runtime          16152 non-null  float64
dtypes: float64(2

In [5]:
def splitter(x):
    try:
        return x.split(",")
    except:
        return []

In [6]:
# Não mexer:
# sorted(list(Counter(rating_db["Directors"].apply(splitter).apply(pd.Series).unstack().reset_index(drop=True).dropna().values).items()),key = lambda x: x[1])

In [7]:
def sorted_bom(x):
    return sorted(list(Counter(rating_db[x].apply(splitter).apply(pd.Series).unstack().reset_index(drop=True).dropna().values).items()),key = lambda x: -x[1])

In [8]:
sorted_bom("Country")

[('United States', 10486),
 ('United Kingdom', 1713),
 ('India', 1157),
 ('Canada', 1117),
 ('France', 741),
 ('Germany', 498),
 ('Italy', 417),
 ('Spain', 329),
 ('Australia', 321),
 ('Hong Kong', 269),
 ('Japan', 266),
 ('China', 189),
 ('Mexico', 183),
 ('South Korea', 162),
 ('Belgium', 151),
 ('Ireland', 128),
 ('Sweden', 118),
 ('Netherlands', 116),
 ('Denmark', 113),
 ('Argentina', 106),
 ('Philippines', 94),
 ('Turkey', 88),
 ('South Africa', 86),
 ('Taiwan', 80),
 ('Israel', 79),
 ('Switzerland', 76),
 ('West Germany', 69),
 ('Thailand', 64),
 ('Brazil', 64),
 ('Poland', 63),
 ('Russia', 62),
 ('New Zealand', 60),
 ('Indonesia', 56),
 ('Norway', 56),
 ('Egypt', 50),
 ('Austria', 48),
 ('Czech Republic', 44),
 ('United Arab Emirates', 43),
 ('Romania', 41),
 ('Finland', 40),
 ('Hungary', 35),
 ('Colombia', 35),
 ('Greece', 32),
 ('Bulgaria', 29),
 ('Singapore', 29),
 ('Luxembourg', 27),
 ('Pakistan', 25),
 ('Iceland', 25),
 ('Nigeria', 25),
 ('Chile', 22),
 ('Portugal', 21),
 (

In [9]:
%%time
# Conversão das classes com apenas uma contribuição para uma categoria "outros":

def generaliza_valores(x, nome_coluna):
    class_per_movie = rating_db[x].apply(splitter).apply(pd.Series)
    all_class_contributions = sorted_bom(x)

    # Classes a serem trocadas
    others = []

    # Identificação das classes com apenas uma contribuição
    
    for classes, contributions in all_class_contributions:
        if int(contributions) < 2:
            others.append(classes)

    # Listando somente classes com mais de 2 contribuições
    official_classes = class_per_movie.replace(others, "Outros "+nome_coluna)
    
    construtor = list(official_classes.columns)
    rating_db[x] = official_classes[construtor].apply(lambda x: ','.join(x[x.notnull()]), axis = 1)

Wall time: 0 ns


In [10]:
%%time
official_languages = generaliza_valores("Language", "Idiomas")

Wall time: 10.4 s


In [11]:
%%time
official_directors = generaliza_valores("Directors", "Diretores")

Wall time: 1min 47s


In [12]:
%%time
official_countries = generaliza_valores("Country", "Paises")

Wall time: 10.7 s


In [14]:
# https://stackoverflow.com/questions/57469676/python-one-hot-encoding-for-comma-separated-values

In [15]:
%%time

new_df = pd.concat([rating_db.drop('Language', 1), rating_db['Language'].str.get_dummies(sep=",")], 1)
print("Rodou 1 coluna")
new_df2 = pd.concat([new_df.drop('Directors', 1), new_df['Directors'].str.get_dummies(sep=",")], 1)
print("Rodou 2 colunas")
new_df3 = pd.concat([new_df2.drop('Genres', 1), new_df2['Genres'].str.get_dummies(sep=",")], 1)
print("Rodou 3 colunas")
rating_db_tratado = pd.concat([new_df3.drop('Country', 1), new_df3['Country'].str.get_dummies(sep=",")], 1)
print("Rodou 4 colunas")

Rodou 1 coluna
Rodou 2 colunas
Rodou 3 colunas
Rodou 4 colunas
Wall time: 42.6 s


In [17]:
# https://stackoverflow.com/questions/37646473/how-could-i-do-one-hot-encoding-with-multiple-values-in-one-cell
# pd.DataFrame({"diretor":pd.Series(['2','1','3']).astype('category',categories = [sorted_bom("Directors")])})

In [18]:
# directors = sorted(list(Counter(official_directors.unstack().reset_index(drop=True).dropna().values).items()),key = lambda x: -x[1])

In [105]:
rating_db_nans = rating_db_tratado.copy()
rating_db_nans = rating_db_nans.dropna(subset=["IMDb"])

In [106]:
rating_db_nans.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16173 entries, 0 to 16742
Columns: 3217 entries, Unnamed: 0 to Zimbabwe
dtypes: float64(2), int64(3212), object(3)
memory usage: 397.1+ MB


In [107]:
from sklearn.impute import SimpleImputer

# Antes de treinar o SimpleImputer, remover a coluna de dados categóricos. O dataset resultante tem apenas
# as variáveis independentes numéricas.
rating_db_imputer = rating_db_nans.copy()

# Cria um imputer que substitui células inválidas (NaN) pela mediana dos valores da coluna à qual a célula pertence.
imputer = SimpleImputer(strategy="median")

# Agora treinar o Imputer. Isto vai causar o cálculo da mediana de cada coluna,
# que ficará armazenado no Imputer para uso futuro.
rating_db_imputer["Runtime"] = imputer.fit_transform(rating_db_imputer[["Runtime"]])

In [108]:
rating_db_nans["Runtime"] = rating_db_imputer["Runtime"]

In [109]:
rating_db_nans[["Runtime"]].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16173 entries, 0 to 16742
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Runtime  16173 non-null  float64
dtypes: float64(1)
memory usage: 252.7 KB


In [110]:
# # Trocar Nan de Age por Desconhecido
# rating_db_age = rating_db_nans.copy()
# rating_db_age = rating_db_age.replace(np.nan, value = "Desconhecido", regex = True)
# # rating_db_nans["Age"] = rating_db_age["Age"]
# rating_db_age[["Age"]].info()

## Regressor

In [99]:
RANDOM_SEED = 42

In [113]:
colunas = ["Rotten Tomatoes", "Age", "Title"]
rating_db_reg = rating_db_nans.copy()
rating_db_reg = rating_db_reg.drop(columns = colunas)

In [114]:
rating_db_reg.head()

,Unnamed: 0,ID,Year,IMDb,Netflix,Hulu,Prime Video,Disney+,Type,Runtime,...,Ukraine,United Arab Emirates,United Kingdom,United States,Uruguay,Venezuela,Vietnam,West Germany,Yugoslavia,Zimbabwe
0,0,1,2010,8.8,1,0,0,0,0,148.0,...,0,0,1,1,0,0,0,0,0,0
1,1,2,1999,8.7,1,0,0,0,0,136.0,...,0,0,0,1,0,0,0,0,0,0
2,2,3,2018,8.5,1,0,0,0,0,149.0,...,0,0,0,1,0,0,0,0,0,0
3,3,4,1985,8.5,1,0,0,0,0,116.0,...,0,0,0,1,0,0,0,0,0,0
4,4,5,1966,8.8,1,0,1,0,0,161.0,...,0,0,0,0,0,0,0,1,0,0


In [115]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(
    rating_db_reg,
    test_size=0.2,
    random_state=RANDOM_SEED,
)

In [116]:
X_train = train_set.drop(columns=["IMDb"])
y_train = train_set["IMDb"]

In [117]:
from sklearn.linear_model import LinearRegression
 
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

LinearRegression()

In [121]:
# Para obter as previsões, basta chamar o método predict()
predicted_labels = lin_reg.predict(X_train)
print("Predição: {}".format(predicted_labels.round(decimals=2)))
# Compare com os valores originais:
print("Original: {}".format(y_train.values.round(decimals=2)))

Predição: [6.06 3.84 5.27 ... 5.99 5.72 5.99]
Original: [6.3 3.2 5.3 ... 5.9 6.  6.2]


In [123]:
from sklearn.metrics import mean_squared_error

predicted_labels = lin_reg.predict(X_train)
lin_mse = mean_squared_error(y_train, predicted_labels)
lin_rmse = np.sqrt(lin_mse)
print("Regressão linear: RMSE = {:.2f}".format(lin_rmse))

Regressão linear: RMSE = 0.79


In [126]:
%%time
from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import StratifiedShuffleSplit

lin_scores = cross_val_score(
    lin_reg,
    X_train,
    y_train,
    scoring="neg_mean_squared_error",
    cv = 10,
    n_jobs=-1)

KeyboardInterrupt: 

In [127]:
lin_rmse_scores = np.sqrt(-lin_scores)

def display_scores(scores):
    print("Scores:", scores.round(decimals=2))
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

display_scores(lin_rmse_scores)

Scores: [1.02000000e+00 1.06700000e+01 1.02000000e+00 1.04000000e+00
 1.01000000e+00 9.90000000e-01 1.90245438e+06 1.02000000e+00
 1.22389000e+03 1.00000000e+00]
Mean: 190369.6049534645
Standard deviation: 570695.0406051836


* Tirar as variáveis mais importantes: como fica o desempenho do regressor?
* Análise exploratória - visualização
* Criar uma lista de perguntas para responder - ex. qual diretor tem mais impacto positivo no rating? Qual a relação entre rating e ano de lançamento?
* Qualidade do código - melhorar nome das variáveis